In [1]:
! pip install tensorflow

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
users=['Mohamed','Karim','Ahmed','Yassine']
courses = [
    'Introduction to 3D Printing','3D Printing with Blender','Introduction to SolidWorks','SolidWorks Simulation Essentials','AutoCAD 3D Modeling','AutoCAD Electrical Essentials'
]
features = ['3D Printing','Solidworks','AutoCad']
num_users=len(users)
num_courses=len(courses)
num_feats=len(features)
num_recommendation = 3

In [4]:
#Each row represents a user rating for the different courses
users_courses=tf.constant([
    [0,5,0,0,4,3],
    [0,0,5,4,4,0],
    [0,0,0,0,5,4],
    [3,0,2,1,0,0]
],dtype=tf.float32)
#Features of the courses one-hot encoded
courses_feats=tf.constant([
    [1,0,0],
    [1,0,0],
    [0,1,0],
    [0,1,0],
    [0,0,1],
    [0,0,1]
],dtype=tf.float32)

# Computing the user feature matrix

In [5]:
users_feats=tf.matmul(users_courses,courses_feats)
users_feats

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[5., 0., 7.],
       [0., 9., 4.],
       [0., 0., 9.],
       [3., 3., 0.]], dtype=float32)>

In [6]:
x=tf.reduce_sum(users_feats,axis=1,keepdims=True)
x

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[12.],
       [13.],
       [ 9.],
       [ 6.]], dtype=float32)>

In [7]:
#Next we normalize each user feature vector to sum to 1. Normalizing isn't strictly neccesary, but it makes it so that rating magnitudes will be comparable between users.
users_feats=users_feats / tf.reduce_sum(users_feats,axis=1,keepdims=True)
users_feats

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0.41666666, 0.        , 0.5833333 ],
       [0.        , 0.6923077 , 0.30769232],
       [0.        , 0.        , 1.        ],
       [0.5       , 0.5       , 0.        ]], dtype=float32)>

# Ranking feature relevance for each user

In [8]:
# indices of the top 3 categorys
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[2, 0, 1],
       [1, 2, 0],
       [2, 0, 1],
       [0, 1, 2]])>

In [9]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], feature_names))

Mohamed: ['AutoCad', '3D Printing', 'Solidworks']
Karim: ['Solidworks', 'AutoCad', '3D Printing']
Ahmed: ['AutoCad', '3D Printing', 'Solidworks']
Yassine: ['3D Printing', 'Solidworks', 'AutoCad']


# Determining course recommendation 

In [10]:
#We will use the dot product as our similarity measure. In essence, this is a weighted movie average for each user.
users_ratings=tf.matmul(users_feats,tf.transpose(courses_feats))

In [11]:
#If a user has already rated a movie, we ignore that rating. 
#This way, we only focus on ratings for previously unseen/unrated courses.
users_unseen_courses = tf.equal(users_courses, tf.zeros_like(users_courses))
rated_courses_mask = tf.greater(users_courses, tf.zeros_like(users_courses))
ignore_matrix = tf.where(rated_courses_mask, tf.ones_like(users_courses)*-np.Inf, tf.zeros_like(users_courses))
users_ratings_new = tf.where(
    users_unseen_courses,
    users_ratings,
    ignore_matrix)


In [12]:
top_courses = tf.nn.top_k(users_ratings_new, num_recommendation)[1]
top_courses

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[0, 2, 3],
       [5, 0, 1],
       [0, 1, 2],
       [1, 4, 5]])>

In [13]:
for i in range(num_users):
    course_names = [courses[index] for index in top_courses[i]]
    print('{}: {}'.format(users[i], course_names))

Mohamed: ['Introduction to 3D Printing', 'Introduction to SolidWorks', 'SolidWorks Simulation Essentials']
Karim: ['AutoCAD Electrical Essentials', 'Introduction to 3D Printing', '3D Printing with Blender']
Ahmed: ['Introduction to 3D Printing', '3D Printing with Blender', 'Introduction to SolidWorks']
Yassine: ['3D Printing with Blender', 'AutoCAD 3D Modeling', 'AutoCAD Electrical Essentials']
